# Quick start to SingleStore using Stages

What we will learn in this notebook
1. Sign up for a free trial
2. Create a database and assign compute resources
3. Loading data using Stages
4. Query data

## Step 1- Sign up for a free trial
1. Go to https://www.singlestore.com/cloud-trial/
2. Just answer a few questions
3. Verify your email

## Step 2- Create a database and assign compute resources


## Step 3- Load data to foodhub_order

This notebook provides step by step instructions for ingesting 'foodhub_order.csv' file to a database.

Start by creating a table to store the ingested data:

In [ ]:
%%sql
# Clean up previously created resources
DROP PIPELINE IF EXISTS foodhub_database.`foodhub_order`;
DROP TABLE IF EXISTS foodhub_database.`foodhub_order`;

USE foodhub_database;
CREATE TABLE foodhub_database.`foodhub_order` (
	`order_id` bigint(20) NULL,
	`customer_id` bigint(20) NULL,
	`restaurant_name` text CHARACTER SET utf8 COLLATE utf8_general_ci NULL,
	`cuisine_type` text CHARACTER SET utf8 COLLATE utf8_general_ci NULL,
	`cost_of_the_order` double NULL,
	`day_of_the_week` text CHARACTER SET utf8 COLLATE utf8_general_ci NULL,
	`rating` text CHARACTER SET utf8 COLLATE utf8_general_ci NULL,
	`food_preparation_time` bigint(20) NULL,
	`delivery_time` bigint(20) NULL,
	 SHARD KEY ()
);

## Create a pipeline that will load file into the database
### [Learn about Load Data with Pipeline through our documentation](https://docs.singlestore.com/managed-service/en/load-data/load-data-with-pipelines.html)

In [ ]:
%%sql
USE foodhub_database;
CREATE PIPELINE foodhub_database.`foodhub_order`
AS LOAD DATA STAGE 'foodhub_order.csv'
BATCH_INTERVAL 2500
DISABLE OUT_OF_ORDER OPTIMIZATION
DISABLE OFFSETS METADATA GC
SKIP DUPLICATE KEY ERRORS -- SKIP ALL ERRORS can be used to skip all errors that can be tracked through "Monitor the pipeline for errors"
INTO TABLE `foodhub_order`
FIELDS TERMINATED BY ',' ENCLOSED BY '"' ESCAPED BY '\\' 
LINES TERMINATED BY '\r\n' STARTING BY '' 
IGNORE 1 LINES
FORMAT CSV
(
	`foodhub_order`.`order_id`,
	`foodhub_order`.`customer_id`,
	`foodhub_order`.`restaurant_name`,
	`foodhub_order`.`cuisine_type`,
	`foodhub_order`.`cost_of_the_order`,
	`foodhub_order`.`day_of_the_week`,
	`foodhub_order`.`rating`,
	`foodhub_order`.`food_preparation_time`,
	`foodhub_order`.`delivery_time`
);

## Start the pipeline

In [ ]:
%%sql
START PIPELINE foodhub_database.`foodhub_order`;

## Monitor the pipeline for errors

In [ ]:
%%sql
USE foodhub_database;
SELECT * FROM information_schema.pipelines_errors
WHERE pipeline_name ='foodhub_order';

## Check that the data has loaded

In [ ]:
%%sql
SELECT * FROM foodhub_database.`foodhub_order`
LIMIT 100;

In [ ]:
%%sql
# Query the data

# What is the average order value?

SELECT AVG (cost_of_the_order)
FROM foodhub_database.`foodhub_order`;

# What is the total number of orders?
SELECT COUNT (order_id)
FROM foodhub_database.`foodhub_order`;


#Using SQrL to generate queries
# I have a table foodhub_database.`foodhub_order` with the fields order_id customer_id restaurant_name cuisine_type	cost_of_the_order. 
Write a query to find out the most popular cuisine type.  

# SQrL created queries
# What is the most popular cuisine?
SELECT cuisine_type, COUNT(*) as orders_count
FROM foodhub_database.foodhub_order
GROUP BY cuisine_type
ORDER BY orders_count DESC LIMIT 1;